In [40]:
import pandas as pd

def get_customer_id(row):
    last_name = str(row['Billing Name']).rsplit(' ', maxsplit=1)[1][:3].upper()
    id_number = str(int(row['Id']))[-5:]
    return last_name + id_number


def get_ship_via(row):
    if row['Shipping Method'] in ["FedEx Overnight (except Puerto Rico)", "FedEx 2nd Day (except Puerto Rico)"] or row['Total'] > 300:
        return '1'
    else:
        return '0'
    

def get_shipping_method(row):
    if row['Ship Via'] == '0':
        return 'US Mail'
    elif row['Ship Via'] == '1':
        return 'UPS'
    else:
        return ''
    

def fix_payment_method(row):
    payment_method = row['Payment Method'].split(' + ')[-1]
    match payment_method:
        case 'Shopify Payments':
            return 'Shopify'
        case 'PayPal Express Checkout':
            return 'PayPal'
        case 'Shop Pay Installments':
            return 'SHOPIFY INSTALLMENTS'
        case _:
            return 'SHOPIFY OTHER'



df = pd.read_csv("O:\Shopify\orders_export.csv", dtype={'Payment Method':str,'Shipping Phone':str}, usecols=['Name','Email','Paid at','Shipping','Taxes','Total','Discount Code','Discount Amount','Shipping Method','Lineitem quantity','Lineitem name','Lineitem price','Lineitem sku','Billing Name','Billing Street','Billing Address1','Billing Address2','Billing City','Billing Zip','Billing Province','Shipping Phone','Shipping Name','Shipping Address1','Shipping Address2','Shipping City','Shipping Zip','Shipping Province','Notes','Payment Method','Id','Risk Level'])
df = df.rename(columns={'Name':'Invoice Number','Paid at':'Date'})
for col in df.columns:
    if df[col].dtype == 'object':
        df[col] = df[col].str.upper()

# merge sales reps by state
reps_by_state = pd.read_csv('Reps by State.csv')
df = df.merge(reps_by_state, how='left', left_on='Shipping Province', right_on='State')

# fillna item level data
fillna_columns = ['Shipping Address2','Billing Address2','Shipping','Taxes','Total','Discount Code','Discount Amount']
for col in fillna_columns:
    if df[col].dtype == "float64":
        df[col].fillna(0, inplace=True)
    else:
        df[col].fillna('', inplace=True)

# ffill order level data
df.sort_values(by='Invoice Number',inplace=True)
ffill_columns = ['Payment Method','Id','Date','Shipping Method','Billing Name','Billing Street','Billing Address1','Billing Address2','Billing City','Billing Zip','Billing Province','Shipping Phone','Shipping Name','Shipping Address1','Shipping Address2','Shipping City','Shipping Zip','Shipping Province']
for col in ffill_columns:
    df[col] = df[col].ffill()


df['Customer ID'] = df.apply(get_customer_id, axis=1)
df['Ship Via'] = df.apply(get_ship_via, axis=1)
df['Customer Payment Method'] = df.apply(fix_payment_method, axis=1)
df['Invoice Number'] = 'S' + df['Invoice Number'].str.strip('#')
df['Shipping Zip'] = df['Shipping Zip'].str.strip("'")
df['Billing Zip'] = df['Billing Zip'].str.strip("'")


In [49]:
df_customers = df[['Customer ID', 'Billing Name', 'Billing Address1', 'Billing Address2', 'Billing City', 'Billing Province', 'Billing Zip', 'Shipping Name', 'Shipping Address1','Shipping Address2','Shipping City','Shipping Province', 'Shipping Zip', 'Shipping Phone', 'Email','Sales Representative ID','Ship Via', 'Customer Payment Method','Discount Code']].copy()
df_customers = df_customers.drop_duplicates(subset='Customer ID')

df_customers['G/L Sales Account'] = '30100'
df_customers['Use Standard Terms'] = 'FALSE'
df_customers['Prepaid Terms'] = 'TRUE'
df_customers['Credit Limit'] = '2500'
df_customers['Use Receipt Settings'] = 'FALSE'
df_customers['Customer Cash Account'] = '10000'

df_customers = df_customers.rename(columns={'Billing Name':'Customer Name',
                                            'Billing Address1':'Bill to Address-Line One',
                                            'Billing Address2':'Bill to Address-Line Two',
                                            'Billing City':'Bill to City',
                                            'Billing Province':'Bill to State',
                                            'Billing Zip':'Bill to Zip',
                                            'Shipping Address1':'Ship to Address 1-Line One',
                                            'Shipping Address2':'Ship to Address 1-Line Two',
                                            'Shipping City':'Ship to City 1',
                                            'Shipping Province':'Ship to State 1',
                                            'Shipping Zip':'Ship to Zipcode 1',
                                            'Shipping Phone':'Telephone 1',
                                            'Email':'Customer E-mail',
                                            'Discount Code':'Prescribing Dr.'})

df_customers = df_customers[['Customer ID','Customer Name','Bill to Address-Line One','Bill to Address-Line Two','Bill to City','Bill to State','Bill to Zip','Ship to Address 1-Line One','Ship to Address 1-Line Two','Ship to City 1','Ship to State 1','Ship to Zipcode 1','Telephone 1','Customer E-mail','Sales Representative ID','G/L Sales Account','Ship Via','Use Standard Terms','Prepaid Terms','Credit Limit','Use Receipt Settings','Customer Payment Method','Customer Cash Account','Prescribing Dr.']]

df_customers.to_csv("O:\Shopify\CUSTOMERS_IMPORT.csv", index=False)

,Customer ID,Customer Name,Bill to Address-Line One,Bill to Address-Line Two,Bill to City,Bill to State,Bill to Zip,Ship to Address 1-Line One,Ship to Address 1-Line Two,Ship to City 1,...,Sales Representative ID,G/L Sales Account,Ship Via,Use Standard Terms,Prepaid Terms,Credit Limit,Use Receipt Settings,Customer Payment Method,Customer Cash Account,Prescribing Dr.
0,GLU86181,LISA GLUCK,818 N CROFT AVE,APT 204,LOS ANGELES,CA,90069,818 N CROFT AVE,APT 204,LOS ANGELES,...,AACTI,30100,1,FALSE,TRUE,2500,FALSE,SHOPIFY OTHER,10000,
1,LAT33893,SAMANTHA LATCHANA,23221 OAKGLEN LANE,,ESTERO,FL,34135,23221 OAKGLEN LANE,,ESTERO,...,AAAMMI,30100,0,FALSE,TRUE,2500,FALSE,SHOPIFY OTHER,10000,KOLKER15
3,LIC36453,REBECCA LICHTENFELD,7 MEARNS WAY,,GREAT BARRINGTON,MA,01230,7 MEARNS WAY,,GREAT BARRINGTON,...,AAESI,30100,0,FALSE,TRUE,2500,FALSE,SHOPIFY OTHER,10000,GAYLE15
